In [1]:
#NOTE: For the code to work, you must change the directory below in the glob.glob line

import requests
import pandas as pd
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr

In [7]:
#This only needs to be executed once, then the files are on your local

yr = 2017

while yr > 1989:
    url = f'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_cbsa_{yr}.zip'
    response = requests.get(url)
    yr -= 1
    with open('temp.zip', 'wb') as f:
        f.write(response.content)
        !unzip 'temp.zip'

'unzip' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


KeyboardInterrupt: 

In [2]:
df = pd.DataFrame()
x = 27

for file in glob.glob('C:/Users/dshaf/Documents/School/classproject/ProjectOne/Project Data/daily*'):

    data = pd.read_csv(file, header = 0)
    df = df.append(data)
    print('file: ' + str(file[-8:-4]) + ', ' + str(x) + ' to go')
    x -= 1

file: 1990, 27 to go
file: 1991, 26 to go
file: 1992, 25 to go
file: 1993, 24 to go
file: 1994, 23 to go
file: 1995, 22 to go
file: 1996, 21 to go
file: 1997, 20 to go
file: 1998, 19 to go
file: 1999, 18 to go
file: 2000, 17 to go
file: 2001, 16 to go
file: 2002, 15 to go
file: 2003, 14 to go
file: 2004, 13 to go
file: 2005, 12 to go
file: 2006, 11 to go
file: 2007, 10 to go
file: 2008, 9 to go
file: 2009, 8 to go
file: 2010, 7 to go
file: 2011, 6 to go
file: 2012, 5 to go
file: 2013, 4 to go
file: 2014, 3 to go
file: 2015, 2 to go
file: 2016, 1 to go
file: 2017, 0 to go


In [3]:
# atlanta_df = df.loc[  (df['CBSA'].apply( lambda s:  "Atlanta" in s  )) & (df['Defining Parameter'] == 'Ozone')]
# dallas_df = df.loc[  (df['CBSA'].apply( lambda s:  "Dallas" in s  )) & (df['Defining Parameter'] == 'Ozone')]
# austin_df = df.loc[  (df['CBSA'].apply( lambda s:  "Austin" in s  )) & (df['Defining Parameter'] == 'Ozone')]
# nashville_df = df.loc[  (df['CBSA'].apply( lambda s:  "Nashville" in s  )) & (df['Defining Parameter'] == 'Ozone')]
combined_df = df.loc[(df['CBSA'].apply( lambda s:  "Atlanta" in s  )) | (df['CBSA'].apply( lambda s:  "Dallas" in s  )) | (df['CBSA'].apply( lambda s:  "Nashville" in s  )) | (df['CBSA'].apply( lambda s:  "Austin" in s  )) ]

In [20]:
combined_df.to_csv('All pollutants Air Quality Data.csv', index = False)

In [4]:
combined_df = combined_df.reset_index()
combined_df['Date'] = combined_df['Date'].astype(str)

lst = combined_df['Date']
year_list = []
for x in lst:
    yr = x[:4]
    year_list.append(yr)
combined_df['Year'] = year_list

In [5]:
combined_df['CBSA'].value_counts()

Nashville-Davidson--Murfreesboro--Franklin, TN    10227
Atlanta-Sandy Springs-Roswell, GA                 10227
Dallas-Fort Worth-Arlington, TX                   10227
Austin-Round Rock, TX                             10225
Name: CBSA, dtype: int64

In [8]:
annual_average = combined_df.groupby(['CBSA', 'Year', 'Defining Parameter'])['AQI'].mean().reset_index()
annual_average.to_csv('AllPollutants_Annual_Avg_by_city.csv', header = True)

In [9]:
annual_average.head()

,CBSA,Year,Defining Parameter,AQI
0,"Atlanta-Sandy Springs-Roswell, GA",1990,CO,38.333333
1,"Atlanta-Sandy Springs-Roswell, GA",1990,NO2,43.767857
2,"Atlanta-Sandy Springs-Roswell, GA",1990,Ozone,119.298246
3,"Atlanta-Sandy Springs-Roswell, GA",1990,PM10,58.400000
4,"Atlanta-Sandy Springs-Roswell, GA",1990,SO2,82.932203


In [10]:
# Get the correlation coefficient of the populations and air quality in each city

#Atlanta
filename = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Census ACS5 Data/Atlanta_Census.csv'

atl_data = pd.read_csv(filename)
atl_data = atl_data.sort_values(['Year'], ascending = True)
atl_pop = atl_data['Population']

atl_ozone_df = annual_average.loc[(annual_average['CBSA'] == 'Atlanta-Sandy Springs-Roswell, GA') & (annual_average['Defining Parameter'] == 'Ozone')]
atl_ozone = atl_ozone_df['AQI'].reset_index()
atl_ozone = atl_ozone.drop(['index'], axis = 1)
atl_ozone = atl_ozone['AQI'][0:27]

atl_corr = pearsonr(atl_pop, atl_ozone)
atl_corco = round(atl_corr[0],2)
atl_p = round(atl_corr[1],2)
print(f'The correlation coefficient between population and ozone in Atlanta is {atl_corco} and the p-value is {atl_p}')

#Austin
filename = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Census ACS5 Data/Austin_Census.csv'

aus_data = pd.read_csv(filename)
aus_data = aus_data.sort_values(['Year'], ascending = True)
aus_pop = aus_data['Population']

aus_ozone_df = annual_average.loc[(annual_average['CBSA'] == 'Austin-Round Rock, TX') & (annual_average['Defining Parameter'] == 'Ozone')]
aus_ozone = aus_ozone_df['AQI'].reset_index()
aus_ozone = aus_ozone.drop(['index'], axis = 1)
aus_ozone = aus_ozone['AQI'][0:27]

aus_corr = pearsonr(aus_pop, aus_ozone)
aus_corco = round(aus_corr[0],2)
aus_p = round(aus_corr[1],2)
print(f'The correlation coefficient between population and ozone in Austin is {aus_corco} and the p-value is {aus_p}')

#Dallas
filename = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Census ACS5 Data/Dallas_Census.csv'

dal_data = pd.read_csv(filename)
dal_data = dal_data.sort_values(['Year'], ascending = True)
dal_pop = dal_data['Population']

dal_ozone_df = annual_average.loc[(annual_average['CBSA'] == 'Austin-Round Rock, TX') & (annual_average['Defining Parameter'] == 'Ozone')]
dal_ozone = dal_ozone_df['AQI'].reset_index()
dal_ozone = dal_ozone.drop(['index'], axis = 1)
dal_ozone = dal_ozone['AQI'][0:27]

dal_corr = pearsonr(dal_pop, dal_ozone)
dal_corco = round(dal_corr[0],2)
dal_p = round(dal_corr[1],2)
print(f'The correlation coefficient between population and ozone in Dallas is {dal_corco} and the p-value is {dal_p}')

#Nashville
filename = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Census ACS5 Data/Nashville_Census.csv'

nash_data = pd.read_csv(filename)
nash_data = nash_data.sort_values(['Year'], ascending = True)
nash_pop = nash_data['Population']

nash_ozone_df = annual_average.loc[(annual_average['CBSA'] == 'Nashville-Davidson--Murfreesboro--Franklin, TN') & (annual_average['Defining Parameter'] == 'Ozone')]
nash_ozone = nash_ozone_df['AQI'].reset_index()
nash_ozone = nash_ozone.drop(['index'], axis = 1)
nash_ozone = nash_ozone['AQI'][0:27]

nash_corr = pearsonr(nash_pop, nash_ozone)
nash_corco = round(nash_corr[0],2)
nash_p = round(nash_corr[1],2)
print(f'The correlation coefficient between population and ozone in Nashville is {nash_corco} and the p-value is {nash_p}')

The correlation coefficient between population and ozone in Atlanta is -0.39 and the p-value is 0.05
The correlation coefficient between population and ozone in Austin is -0.11 and the p-value is 0.58
The correlation coefficient between population and ozone in Dallas is -0.1 and the p-value is 0.62
The correlation coefficient between population and ozone in Nashville is -0.75 and the p-value is 0.0


In [13]:
# Get the correlation coefficient b/w the manufacturing data and air quality in each city

#Atlanta
filename_man = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Manufacturing Data/Atlanta.csv'

atl_man_data = pd.read_csv(filename_man)
atl_man = atl_man_data['Annual'][0:27]
atl_man_corr = pearsonr(atl_man, atl_ozone)
atl_man_corco = round(atl_man_corr[0],2)
atl_m = round(atl_man_corr[1],2)
print(f'The correlation coefficient between manufacturing and ozone in Atlanta is {atl_man_corco} and the p-value is {atl_m}')

#Austin
filename_man = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Manufacturing Data/Austin.csv'

aus_man_data = pd.read_csv(filename_man)
aus_man = aus_man_data['Annual'][0:27]
aus_man_corr = pearsonr(aus_man, aus_ozone)
aus_man_corco = round(aus_man_corr[0],2)
aus_m = round(aus_man_corr[1],2)
print(f'The correlation coefficient between manufacturing and ozone in Austin is {aus_man_corco} and the p-value is {aus_m}')

#Dallas
filename_man = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Manufacturing Data/Dallas.csv'

dal_man_data = pd.read_csv(filename_man)
dal_man = dal_man_data['Annual'][0:27]
dal_man_corr = pearsonr(dal_man, dal_ozone)
dal_man_corco = round(dal_man_corr[0],2)
dal_m = round(dal_man_corr[1],2)
print(f'The correlation coefficient between manufacturing and ozone in Dallas is {dal_man_corco} and the p-value is {dal_m}')

#Nashville
filename_man = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Manufacturing Data/Nashville.csv'

nash_man_data = pd.read_csv(filename_man)
nash_man = nash_man_data['Annual'][0:27]
nash_man_corr = pearsonr(nash_man, nash_ozone)
nash_man_corco = round(nash_man_corr[0],2)
nash_m = round(nash_man_corr[1],2)
print(f'The correlation coefficient between manufacturing and ozone in Nashville is {nash_man_corco} and the p-value is {nash_m}')

The correlation coefficient between manufacturing and ozone in Atlanta is 0.73 and the p-value is 0.0
The correlation coefficient between manufacturing and ozone in Austin is 0.5 and the p-value is 0.01
The correlation coefficient between manufacturing and ozone in Dallas is 0.34 and the p-value is 0.08
The correlation coefficient between manufacturing and ozone in Nashville is 0.71 and the p-value is 0.0


In [19]:
# Get the correlation coefficient b/w the vehicle data and air quality in each city

filename_vehicle = 'C:/Users/dshaf/Documents/School/classproject/ProjectOne/Manufacturing Data/Vehicles available for work commute by Metro Area.xlsx'
vehicle_data = pd.read_excel(filename_vehicle)
vehicle_data = vehicle_data.rename(columns = {'Aggregate nunber or vehicles ailable for work commute':'vehicle_count'})

#Atlanta

atl_veh_data = vehicle_data.loc[vehicle_data['Metro Area'] == 12060]
atl_veh = atl_veh_data['vehicle_count'][0:8]
atl_veh_corr = pearsonr(atl_veh, atl_ozone[0:8])
atl_veh_corco = round(atl_veh_corr[0],2)
atl_v = round(atl_veh_corr[1],2)
print(f'The correlation coefficient between vehicles and ozone in Atlanta is {atl_veh_corco} and the p-value is {atl_v}')

#Austin

aus_veh_data = vehicle_data.loc[vehicle_data['Metro Area'] == 12420]
aus_veh = aus_veh_data['vehicle_count'][0:8]
aus_veh_corr = pearsonr(aus_veh, aus_ozone[0:8])
aus_veh_corco = round(aus_veh_corr[0],2)
aus_v = round(aus_veh_corr[1],2)
print(f'The correlation coefficient between vehicles and ozone in Austin is {aus_veh_corco} and the p-value is {aus_v}')

#Dallas

dal_veh_data = vehicle_data.loc[vehicle_data['Metro Area'] == 19100]
dal_veh = dal_veh_data['vehicle_count'][0:8]
dal_veh_corr = pearsonr(dal_veh, dal_ozone[0:8])
dal_veh_corco = round(dal_veh_corr[0],2)
dal_v = round(dal_veh_corr[1],2)
print(f'The correlation coefficient between vehicles and ozone in Dallas is {dal_veh_corco} and the p-value is {dal_v}')

#Nashville

nash_veh_data = vehicle_data.loc[vehicle_data['Metro Area'] == 34980]
nash_veh = nash_veh_data['vehicle_count'][0:8]
nash_veh_corr = pearsonr(nash_veh, nash_ozone[0:8])
nash_veh_corco = round(nash_veh_corr[0],2)
nash_v = round(nash_veh_corr[1],2)
print(f'The correlation coefficient between vehicles and ozone in Nashville is {nash_veh_corco} and the p-value is {nash_v}')

The correlation coefficient between vehicles and ozone in Atlanta is 0.28 and the p-value is 0.49
The correlation coefficient between vehicles and ozone in Austin is -0.01 and the p-value is 0.98
The correlation coefficient between vehicles and ozone in Dallas is 0.01 and the p-value is 0.98
The correlation coefficient between vehicles and ozone in Nashville is -0.47 and the p-value is 0.24


In [ ]:
#Read annual average air quality data into Pandas

atl_aq = pd.read_csv("C:/Users/jessi/Documents/ProjectOne/Annual_Averages_by_cityAtl.csv")
# atl_aq.head()

aus_aq = pd.read_csv("C:/Users/jessi/Documents/ProjectOne/Annual_Averages_by_cityAus.csv")
# aus_aq.head()

dal_aq = pd.read_csv("C:/Users/jessi/Documents/ProjectOne/Annual_Averages_by_cityDal.csv")
# dal_aq.head()

nash_aq = pd.read_csv("C:/Users/jessi/Documents/ProjectOne/Annual_Averages_by_cityNash.csv")
# nash_aq.head()

In [ ]:
x_axis = np.arange(len(atl_aq["Year"]))
tick_locations = [value+0.4 for value in x_axis] 

fig, ax1 = plt.subplots(figsize=(20,6))

ax1.set_xlabel("Year")
ax1.set_ylabel("Air Quality")
Atlanta, = ax1.plot(x_axis, atl_aq["AQI"], color='b',label="Atlanta")
ax1.tick_params(axis='y', labelcolor='k')
Year = ax1.set_xticks(list(range(0, len(atl_aq["Year"]))))
ax1.set_xticklabels(atl_aq["Year"], rotation="vertical")
# ax1.axvline(0, ymin=0, ymax=1, color='g')
ax1.axvline(6, ymin=0, ymax=1, color='g')
ax1.axvline(8, ymin=0, ymax=1, color='g')
ax1.axvline(12, ymin=0, ymax=1, color='g')
ax1.axvline(15, ymin=0, ymax=1, color='g')
ax1.axvline(25, ymin=0, ymax=1, color='r')

# plt.legend(handles=[Atlanta], loc="best")

plt.title("Atlanta Air Quality")

plt.savefig("C:/Users/jessi/Documents/ProjectOne/Atlanta Air Quality Comparison.png")

In [ ]:
fig, ax1 = plt.subplots(figsize=(20,6))

ax1.set_xlabel("Year")
ax1.set_ylabel("Air Quality")
Austin, = ax1.plot(x_axis, aus_aq["AQI"], color='k',label="Austin")
ax1.tick_params(axis='y', labelcolor='k')
Year = ax1.set_xticks(list(range(0, len(atl_aq["Year"]))))
ax1.set_xticklabels(atl_aq["Year"], rotation="vertical")
# ax1.axvline(0, ymin=0, ymax=1, color='g')
ax1.axvline(0, ymin=0, ymax=1, color='g')
ax1.axvline(5, ymin=0, ymax=1, color='r')
ax1.axvline(11, ymin=0, ymax=1, color='g')
ax1.axvline(15, ymin=0, ymax=1, color='g')
ax1.axvline(19, ymin=0, ymax=1, color='g')
ax1.axvline(23, ymin=0, ymax=1, color='g')
ax1.axvline(25, ymin=0, ymax=1, color='r')

# plt.legend(handles=[Atlanta], loc="best")

plt.title("Austin Air Quality")

plt.savefig("C:/Users/jessi/Documents/ProjectOne/Austin Air Quality Comparison.png")

In [ ]:
fig, ax1 = plt.subplots(figsize=(20,6))

ax1.set_xlabel("Year")
ax1.set_ylabel("Air Quality")
Dallas, = ax1.plot(x_axis, dal_aq["AQI"], color='k',label="Dallas")
ax1.tick_params(axis='y', labelcolor='k')
Year = ax1.set_xticks(list(range(0, len(atl_aq["Year"]))))
ax1.set_xticklabels(atl_aq["Year"], rotation="vertical")
# ax1.axvline(0, ymin=0, ymax=1, color='g')
ax1.axvline(0, ymin=0, ymax=1, color='g')
ax1.axvline(5, ymin=0, ymax=1, color='r')
ax1.axvline(11, ymin=0, ymax=1, color='g')
ax1.axvline(12, ymin=0, ymax=1, color='g')
ax1.axvline(19, ymin=0, ymax=1, color='g')
ax1.axvline(23, ymin=0, ymax=1, color='g')
ax1.axvline(25, ymin=0, ymax=1, color='r')

# plt.legend(handles=[Atlanta], loc="best")

plt.title("Dallas Air Quality")

plt.savefig("C:/Users/jessi/Documents/ProjectOne/Dallas Air Quality Comparison.png")

In [ ]:
fig, ax1 = plt.subplots(figsize=(20,6))

ax1.set_xlabel("Year")
ax1.set_ylabel("Air Quality")
Nashville, = ax1.plot(x_axis, nash_aq["AQI"], color='k',label="Nashville")
ax1.tick_params(axis='y', labelcolor='k')
Year = ax1.set_xticks(list(range(0, len(atl_aq["Year"]))))
ax1.set_xticklabels(atl_aq["Year"], rotation="vertical")
# ax1.axvline(0, ymin=0, ymax=1, color='g')
ax1.axvline(6, ymin=0, ymax=1, color='g')
ax1.axvline(8, ymin=0, ymax=1, color='g')
ax1.axvline(20, ymin=0, ymax=1, color='g')
ax1.axvline(22, ymin=0, ymax=1, color='g')

# plt.legend(handles=[Atlanta], loc="best")

plt.title("Nashville Air Quality")

plt.savefig("C:/Users/jessi/Documents/ProjectOne/Nashville Air Quality Comparison.png")

In [ ]:
#Air Quality comparison between cities

x_axis = np.arange(len(atl_aq["Year"]))
tick_locations = [value+0.4 for value in x_axis]
data2 = [139, 142, 151, 162, 172, 182, 186, 184, 177, 165, 155, 144, 
         139, 142, 151, 162, 172, 182, 186, 184, 177, 165, 155, 144, 
         89, 162, 172, 182, 73] 